# Simulation study - Inference of tumour growth model in a virtual patient without treatment 

## Build tumour growth model

In [1]:
import erlotinib as erlo

# Define mechanistic model
path = erlo.ModelLibrary().tumour_growth_inhibition_model_koch_reparametrised()
mechanistic_model = erlo.PharmacodynamicModel(path)
mechanistic_model.set_parameter_names(names={
    'myokit.tumour_volume': 'Tumour volume in cm^3',
    'myokit.critical_volume': 'Critical volume in cm^3',
    'myokit.drug_concentration': 'Drug concentration in mg/L',
    'myokit.kappa': 'Potency in L/mg/day',
    'myokit.lambda': 'Exponential growth rate in 1/day'})

# Define error model
error_model = erlo.ConstantAndMultiplicativeGaussianErrorModel()

# Build model
problem = erlo.ProblemModellingController(
    mechanistic_model, error_model)
problem.fix_parameters({
    'Drug concentration in mg/L': 0,
    'Potency in L/mg/day': 0})

## Measure tumour volume in a virtual patient

In [2]:
import numpy as np

# Get predictive model
predictive_model = problem.get_predictive_model()

# Define a virtual patient
parameters = [
    0.22,  # Initial tumour volume
    3.76,  # Critical tumour volume
    0.13,  # Tumour growth rate
    0.04,  # Base noise
    0.1]   # Relative noise

# Take virtual measurements
seed = 42
times = np.linspace(0, 30, num=10)
n_samples = 1
data = predictive_model.sample(parameters, times, n_samples, seed)

### Visualise virtual measurements

In [3]:
# Create scatter plot
fig = erlo.plots.PDTimeSeriesPlot()
fig.add_data(data, biomarker='myokit.tumour_volume', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')

# Show figure
fig.show()

**Figure 1:** Visualisation of the measured tumour growth in a virtual patient.

## Build prior predictive model

In [4]:
import pints

# Define prior distribution
log_priors = [
    pints.HalfCauchyLogPrior(location=1, scale=1),     # Initial tumour volume
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Critical tumour volume
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Growth rate
    pints.HalfCauchyLogPrior(location=0, scale=1),     # Sigma base
    pints.HalfCauchyLogPrior(location=0, scale=0.1)]   # Sigma rel.
log_prior = pints.ComposedLogPrior(*log_priors)

# Define prior predictive model and sample 1000 virtual patients
model = erlo.PriorPredictiveModel(predictive_model, log_prior)
n_samples = 1000
samples = model.sample(times, n_samples, seed)

# Visualise prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 2:** Prior predictive model of tumour volume measurement over time. The shaded areas estimate the 0.3, 0.6 and 0.9 bulk probability of the prior predictive distribution by sampling 1000 virtual "measurements" at each of the time points.

## Find MAP estimates for model parameters

In [5]:
# Create posterior
problem.set_data(
    data, meas_key='Sample', dose_key=None, dose_duration_key=None)
problem.set_log_prior(log_priors)
log_posterior = problem.get_log_posteriors()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posterior)
opt.set_transform(transform=pints.LogTransformation(n_parameters=5))
map_estimates = opt.run(show_run_progress_bar=True)

### Visualise optimisation results

In [6]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()

**Figure 3:** Maximum a posteriori (MAP) estimates of the model parameters. The y axis displays the estimated parameter value, and the x axis the corresponding individual.

## Find posterior probability distribution

In [7]:
# Set up sampling controller
sampler = erlo.SamplingController(log_posterior)
sampler.set_initial_parameters(data=map_estimates)
sampler.set_transform(transform=pints.LogTransformation(n_parameters=5))

# Run sampling
posterior_samples = sampler.run(n_iterations=4000, show_progress_bar=True)

### Visualise posterior samples

In [8]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=posterior_samples, warm_up_iter=2000)

fig.show()

**Figure 4:** Marginal posterior distributions of model parameters. The y axis displays the sampled parameter value, and the x axis the binned number of samples for each individual.

## Posterior predictive checks

In [9]:
model = erlo.PosteriorPredictiveModel(predictive_model, posterior_samples)
samples = model.sample(times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 5:** Posterior predictive model of future measurements of the virtual patient (ID 1). The shaded area illustrates the approximate 90% bulk probability of the posterior predictive model constructed from 1000 samples from the posterior distribution and subsequent virtual "measurements" for each of 50 equidistant time points. The circles represent the measurements of the virtual patient that were used to infer the posterior distirbution.

### MAP predictive model (just for comparison)

In [10]:
# Sample from predictive model
times = np.linspace(0, 30)
parameters = [0.24, 4.16, 0.12, 0.03, 0.06]
n_samples = 1000
samples = predictive_model.sample(parameters, times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, biomarker='myokit.tumour_volume', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 6:** Predictive model constructed from the maximum a posteriori (MAP) estimates for the virtual patient (ID 1). The shaded area illustrates the approximate 90% bulk probability of the predictive model constructed from 1000 samples for each of 50 equidistant time points, and the circles represent the measurements of the virtual patient.